<a href="https://colab.research.google.com/github/selu01/housing_production/blob/main/Copy_of_gradio_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install -q gradio scikit-learn joblib pandas

import gradio as gr
import joblib
import numpy as np
import pandas as pd
import urllib.request
import os

print("🚀 Setting up California Housing Predictor...")

# Download and load model
model_path = "forest_model.pkl"
if not os.path.exists(model_path):
    print("📥 Downloading model...")
    url = "https://huggingface.co/JuliaLenc/california-housing-model/resolve/main/forest_model.pkl"
    urllib.request.urlretrieve(url, model_path)

model = joblib.load(model_path)
print("✅ Model ready!")

# Prediction function with beautiful formatting
def predict_price(longitude, latitude, housing_median_age, total_rooms,
                  total_bedrooms, population, households, median_income, ocean_proximity):
    try:
        if households == 0 or total_rooms == 0:
            return "# ❌ Error\nHouseholds and Total Rooms must be greater than 0", ""

        # Engineer features
        rooms_per_house = total_rooms / households
        bedrooms_ratio = total_bedrooms / total_rooms
        people_per_house = population / households

        # Create input
        X = pd.DataFrame([[
            longitude, latitude, housing_median_age, total_rooms,
            total_bedrooms, population, households, median_income,
            rooms_per_house, bedrooms_ratio, people_per_house, ocean_proximity
        ]], columns=[
            'longitude', 'latitude', 'housing_median_age', 'total_rooms',
            'total_bedrooms', 'population', 'households', 'median_income',
            'rooms_per_house', 'bedrooms_ratio', 'people_per_house', 'ocean_proximity'
        ])

        # Predict
        prediction = model.predict(X)[0]

        # Beautiful main output
        main_output = f"""
# 💰 ${prediction:,.0f}
### Predicted Median House Value
"""

        # Detailed stats output
        details_output = f"""
### 📊 Engineered Features

| Feature | Value |
|---------|-------|
| 🏠 Rooms per House | {rooms_per_house:.2f} |
| 🛏️ Bedrooms Ratio | {bedrooms_ratio:.3f} |
| 👥 People per House | {people_per_house:.2f} |

### 📍 Location Details
- **Coordinates**: ({longitude:.2f}, {latitude:.2f})
- **Ocean Proximity**: {ocean_proximity}
- **Median Age**: {housing_median_age} years
- **Median Income**: ${median_income * 10000:,.0f}/year
"""

        return main_output, details_output

    except Exception as e:
        return f"# ❌ Error\n{str(e)}", ""

# Create beautiful interface with custom CSS
custom_css = """
.prediction-box {
    font-size: 48px !important;
    font-weight: bold !important;
    color: #2563eb !important;
    text-align: center !important;
    padding: 30px !important;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    border-radius: 15px !important;
    color: white !important;
}
.gradio-container {
    font-family: 'Inter', sans-serif !important;
}
"""

with gr.Blocks(css=custom_css, title="🏡 California Housing Predictor", theme=gr.themes.Soft()) as demo:

    gr.Markdown("""
    # 🏡 California Housing Price Predictor
    ### Use machine learning to predict median house values in California districts
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📍 Location")
            longitude = gr.Number(label="Longitude", value=-122.23, info="Range: -124 to -114")
            latitude = gr.Number(label="Latitude", value=37.88, info="Range: 32 to 42")
            ocean_proximity = gr.Dropdown(
                label="Ocean Proximity",
                choices=['<1H OCEAN', 'INLAND', 'NEAR BAY', 'NEAR OCEAN', 'ISLAND'],
                value='NEAR BAY',
                info="Distance to ocean"
            )

            gr.Markdown("### 🏘️ District Characteristics")
            housing_median_age = gr.Slider(
                label="Housing Median Age (years)",
                minimum=1, maximum=52, value=41, step=1
            )
            median_income = gr.Slider(
                label="Median Income ($10k units)",
                minimum=0.5, maximum=15, value=8.3252, step=0.1,
                info="Multiply by 10,000 for actual income"
            )

            gr.Markdown("### 🏠 Housing Stats")
            total_rooms = gr.Number(label="Total Rooms", value=880)
            total_bedrooms = gr.Number(label="Total Bedrooms", value=129)
            households = gr.Number(label="Households", value=126)
            population = gr.Number(label="Population", value=322)

        with gr.Column(scale=1):
            gr.Markdown("### 🎯 Prediction Result")
            prediction_output = gr.Markdown(elem_classes="prediction-box")
            details_output = gr.Markdown()

    predict_btn = gr.Button("🔮 Predict House Value", variant="primary", size="lg")

    gr.Markdown("---")
    gr.Markdown("### 💡 Try These Example Districts:")

    gr.Examples(
        examples=[
            [-122.23, 37.88, 41, 880, 129, 322, 126, 8.3252, 'NEAR BAY'],     # High value
            [-119.01, 36.06, 25, 1450, 235, 558, 219, 5.6431, 'INLAND'],      # Mid value
            [-117.96, 33.89, 17, 2853, 470, 1650, 458, 3.1250, '<1H OCEAN'],  # Coastal
            [-121.97, 37.35, 30, 1200, 200, 400, 150, 6.5000, 'NEAR BAY'],    # Silicon Valley
        ],
        inputs=[longitude, latitude, housing_median_age, total_rooms,
                total_bedrooms, population, households, median_income, ocean_proximity],
        label=None
    )

    predict_btn.click(
        fn=predict_price,
        inputs=[longitude, latitude, housing_median_age, total_rooms,
                total_bedrooms, population, households, median_income, ocean_proximity],
        outputs=[prediction_output, details_output]
    )

    gr.Markdown("""
    ---
    <div style='text-align: center; color: #666; font-size: 14px;'>
    <b>Model:</b> Random Forest Regressor |
    <b>Features:</b> 12 (8 raw + 3 engineered + 1 categorical) |
    <b>Dataset:</b> California Housing (1990 Census)
    </div>
    """)

print("🚀 Launching app...")
demo.launch(share=True)
print("✅ App is live!")


🚀 Setting up California Housing Predictor...
📥 Downloading model...
✅ Model ready!


/tmp/ipython-input-474764827.py:94: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, title="🏡 California Housing Predictor", theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-474764827.py:94: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, title="🏡 California Housing Predictor", theme=gr.themes.Soft()) as demo:


🚀 Launching app...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://32253dc00b901ba684.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ App is live!
